Позаимствовали агрегированные фичи у nazarovo. Спасибо! <br>
https://github.com/nazarovo/data-fusion-2024-competitions2

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, CatBoostClassifier, Pool

In [2]:
train = pd.read_csv('../data/train.csv').set_index("user_id")
train

,target,time
user_id,,
3,0,77
13,0,86
37,0,89
41,0,57
42,0,84
...,...,...
561824,0,91
562043,0,75
562312,0,91


In [3]:
report_dates = pd.read_csv('../data/report_dates.csv', parse_dates=['report_dt']).set_index("report")
report_dates["report_dt"] = report_dates["report_dt"].astype('datetime64[ns]')
report_dates

,report_dt
report,
1,2022-07-31 03:00:00
2,2022-08-31 03:00:00
3,2022-09-30 03:00:00
4,2022-10-31 03:00:00
5,2022-11-30 03:00:00
6,2022-12-31 03:00:00
7,2023-01-31 03:00:00
8,2023-02-28 03:00:00
9,2023-03-31 03:00:00


In [4]:
%%time

clients = pd.read_csv('../data/clients.csv').set_index("user_id")
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':0,
                                      'ОТ 11 ДО 50':2,'ОТ 51 ДО 100':3,'БОЛЕЕ 500':5,'ОТ 11 ДО 30':2,
                                      'ОТ 31 ДО 50':2}}, inplace=True)
clients = clients.join(report_dates, on="report", how="left")
clients

CPU times: user 163 ms, sys: 16.7 ms, total: 180 ms
Wall time: 179 ms


,report,employee_count_nm,bankemplstatus,customer_age,report_dt
user_id,,,,,
3,2,4.0,0,3,2022-08-31 03:00:00
9,1,6.0,0,3,2022-07-31 03:00:00
13,6,5.0,0,2,2022-12-31 03:00:00
37,5,6.0,0,2,2022-11-30 03:00:00
41,1,4.0,0,2,2022-07-31 03:00:00
...,...,...,...,...,...
562043,12,NaN,0,2,2023-06-30 03:00:00
562205,12,NaN,0,1,2023-06-30 03:00:00
562312,12,NaN,0,0,2023-06-30 03:00:00


In [5]:
%%time
transactions = pd.read_csv('../data/transactions.csv', 
                           parse_dates=['transaction_dttm'], )
                           #low_memory=False, compression='zip')
transactions

CPU times: user 20.8 s, sys: 2.1 s, total: 22.9 s
Wall time: 22.8 s


,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,3,3,1,-183.883957,2022-01-28 12:05:33
1,3,3,1,-3206.437012,2022-01-28 12:52:30
2,3,16,1,-153866.890625,2022-02-16 14:45:56
3,3,56,1,-15144.601562,2022-03-09 19:58:29
4,3,0,1,5297.908691,2022-03-12 18:11:31
...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20 11:52:09
13075019,562740,9,1,-187.658463,2023-03-20 12:10:22
13075020,562740,1,1,-891.933350,2023-03-20 15:53:37
13075021,562740,13,1,-464.467316,2023-03-20 15:54:49


In [6]:
def logsumabs(x):
    return (np.log(x.abs().sum()))
temp_tran = transactions.groupby(['user_id','currency_rk'])['transaction_amt'].agg([logsumabs])
temp_tran = temp_tran.unstack('currency_rk').fillna(0)
temp_tran.columns = ['logsumabs_0','logsumabs_1','logsumabs_2','logsumabs_3']
temp_tran

,logsumabs_0,logsumabs_1,logsumabs_2,logsumabs_3
user_id,,,,
3,0.00000,12.789712,0.0,0.0
9,0.00000,12.686752,0.0,0.0
13,9.28478,12.825245,0.0,0.0
37,0.00000,12.775178,0.0,0.0
41,0.00000,11.595303,0.0,0.0
...,...,...,...,...
562043,0.00000,10.294896,0.0,0.0
562205,0.00000,10.701664,0.0,0.0
562312,0.00000,9.827568,0.0,0.0


In [7]:
%%time
transactions['transaction_dttm'] = transactions['transaction_dttm'].astype('datetime64[ns]')
transactions['dweek'] = transactions['transaction_dttm'].dt.dayofweek# .day//8
transactions['date'] = transactions['transaction_dttm'].dt.date.astype('datetime64[ns]') # .day//8
transactions['sp'] = np.where( transactions['transaction_amt']>0,transactions['transaction_amt'],0)
transactions['sm'] = np.where( transactions['transaction_amt']<0,transactions['transaction_amt'],0)
transactions['ss'] = transactions['sp'] - transactions['sm'] 
transactions

CPU times: user 16.3 s, sys: 642 ms, total: 16.9 s
Wall time: 16.9 s


,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,dweek,date,sp,sm,ss
0,3,3,1,-183.883957,2022-01-28 12:05:33,4,2022-01-28,0.000000,-183.883957,183.883957
1,3,3,1,-3206.437012,2022-01-28 12:52:30,4,2022-01-28,0.000000,-3206.437012,3206.437012
2,3,16,1,-153866.890625,2022-02-16 14:45:56,2,2022-02-16,0.000000,-153866.890625,153866.890625
3,3,56,1,-15144.601562,2022-03-09 19:58:29,2,2022-03-09,0.000000,-15144.601562,15144.601562
4,3,0,1,5297.908691,2022-03-12 18:11:31,5,2022-03-12,5297.908691,0.000000,5297.908691
...,...,...,...,...,...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20 11:52:09,0,2023-03-20,0.000000,-2484.366211,2484.366211
13075019,562740,9,1,-187.658463,2023-03-20 12:10:22,0,2023-03-20,0.000000,-187.658463,187.658463
13075020,562740,1,1,-891.933350,2023-03-20 15:53:37,0,2023-03-20,0.000000,-891.933350,891.933350
13075021,562740,13,1,-464.467316,2023-03-20 15:54:49,0,2023-03-20,0.000000,-464.467316,464.467316


In [8]:
transactions = transactions.join(clients ,on="user_id", how="left")
transactions['diff_days'] = (transactions['report_dt']-transactions['transaction_dttm']).dt.days
transactions['m'] = (transactions['report_dt'].dt.year-
                     transactions['transaction_dttm'].dt.year)*12+(transactions['report_dt'].dt.month-
                                                                   transactions['transaction_dttm'].dt.month)
transactions

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,dweek,date,sp,sm,ss,report,employee_count_nm,bankemplstatus,customer_age,report_dt,diff_days,m
0,3,3,1,-183.883957,2022-01-28 12:05:33,4,2022-01-28,0.000000,-183.883957,183.883957,2,4.0,0,3,2022-08-31 03:00:00,214,7
1,3,3,1,-3206.437012,2022-01-28 12:52:30,4,2022-01-28,0.000000,-3206.437012,3206.437012,2,4.0,0,3,2022-08-31 03:00:00,214,7
2,3,16,1,-153866.890625,2022-02-16 14:45:56,2,2022-02-16,0.000000,-153866.890625,153866.890625,2,4.0,0,3,2022-08-31 03:00:00,195,6
3,3,56,1,-15144.601562,2022-03-09 19:58:29,2,2022-03-09,0.000000,-15144.601562,15144.601562,2,4.0,0,3,2022-08-31 03:00:00,174,5
4,3,0,1,5297.908691,2022-03-12 18:11:31,5,2022-03-12,5297.908691,0.000000,5297.908691,2,4.0,0,3,2022-08-31 03:00:00,171,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20 11:52:09,0,2023-03-20,0.000000,-2484.366211,2484.366211,12,NaN,0,0,2023-06-30 03:00:00,101,3
13075019,562740,9,1,-187.658463,2023-03-20 12:10:22,0,2023-03-20,0.000000,-187.658463,187.658463,12,NaN,0,0,2023-06-30 03:00:00,101,3
13075020,562740,1,1,-891.933350,2023-03-20 15:53:37,0,2023-03-20,0.000000,-891.933350,891.933350,12,NaN,0,0,2023-06-30 03:00:00,101,3
13075021,562740,13,1,-464.467316,2023-03-20 15:54:49,0,2023-03-20,0.000000,-464.467316,464.467316,12,NaN,0,0,2023-06-30 03:00:00,101,3


In [9]:
count_trans = transactions.groupby(['user_id']
    ).agg({
        'transaction_amt':[
            "sum",
            "max",
            "min",
          #  "median",
            "count",
            #"last"
            ],
        "mcc_code":[
            #"last",
            "nunique"
            ],
        "currency_rk":[
            "min",
            "max"
            ],
        'date':[
            "min",
            "max",
            "count"
            ],
        "sp":"sum",
        "sm":"sum",
        "ss":"sum"
        })
count_trans.columns = count_trans.columns.map('_'.join).map(lambda x: "count_trans_"+str(x))
count_trans["count_trans_dates"]=(count_trans["count_trans_date_max"]-count_trans["count_trans_date_min"]).dt.days
count_trans

,count_trans_transaction_amt_sum,count_trans_transaction_amt_max,count_trans_transaction_amt_min,count_trans_transaction_amt_count,count_trans_mcc_code_nunique,count_trans_currency_rk_min,count_trans_currency_rk_max,count_trans_date_min,count_trans_date_max,count_trans_date_count,count_trans_sp_sum,count_trans_sm_sum,count_trans_ss_sum,count_trans_dates
user_id,,,,,,,,,,,,,,
3,13706.416641,104011.960938,-153866.890625,11,4,1,1,2022-01-28,2022-05-14,11,186108.229797,-172401.813156,358510.042953,106
9,-323434.666813,-45.579891,-90147.617188,90,22,1,1,2021-10-20,2022-04-19,90,0.000000,-323434.666813,323434.666813,181
13,-124717.379150,70322.828125,-58740.300781,22,4,0,1,2022-03-23,2022-09-07,22,128766.684326,-253484.063477,382250.747803,168
37,-331859.599463,5487.140625,-35782.984375,315,28,1,1,2022-02-19,2022-08-17,315,10738.788574,-342598.388037,353337.176611,179
41,-108586.614166,-290.766998,-16841.208984,16,5,1,1,2021-11-16,2022-04-18,16,0.000000,-108586.614166,108586.614166,153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,-29581.256115,-40.769005,-13020.519531,37,12,1,1,2022-10-06,2023-02-07,37,0.000000,-29581.256115,29581.256115,124
562205,-40491.908630,1595.461060,-6220.171387,151,15,1,1,2022-09-22,2023-03-20,151,1968.906334,-42460.814964,44429.721298,179
562312,-18537.821270,-28.292030,-1372.377075,56,7,1,1,2022-09-22,2023-02-27,56,0.000000,-18537.821270,18537.821270,158


In [10]:
# Сумма положительный транзакций
sum_trans_p = transactions[transactions['transaction_amt']>0].groupby(['user_id']
    ).agg({'transaction_amt':["count","mean"]})#.rename(columns={"transaction_amt":"sum_trans_p"})
sum_trans_p.columns = sum_trans_p.columns.map('_'.join).map(lambda x: "sump_"+str(x))
sum_trans_p

,sump_transaction_amt_count,sump_transaction_amt_mean
user_id,,
3,7,26586.889971
13,8,16095.835541
37,2,5369.394287
42,11,6616.334467
46,6,901.388145
...,...,...
561824,4,484.960381
561908,10,93679.223340
562205,3,656.302111


In [11]:
# Сумма отрицательных транзакций
sum_trans_m = transactions[transactions['transaction_amt']<0].groupby(['user_id']
    ).agg({'transaction_amt':["count","mean"]})#.rename(columns={"transaction_amt":"sum_trans_p"})
sum_trans_m.columns = sum_trans_m.columns.map('_'.join).map(lambda x: "summ_"+str(x))
sum_trans_m

,summ_transaction_amt_count,summ_transaction_amt_mean
user_id,,
3,4,-43100.453289
9,90,-3593.718520
13,14,-18106.004534
37,313,-1094.563540
41,16,-6786.663385
...,...,...
562043,37,-799.493409
562205,148,-286.897398
562312,56,-331.032523


In [12]:
msump = transactions.pivot_table(values="sp",                              
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msump.columns = list( map(lambda x: "msump"+str(x), msump.columns ) )
msump

,msump3,msump4,msump5,msump6,msump7,msump8,msump9
user_id,,,,,,,
3,137734.573242,43075.747864,5297.908691,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13,10805.421875,10952.112305,0.000000,86198.161377,10038.188965,10772.799805,0.000000
37,0.000000,0.000000,5487.140625,5251.647949,0.000000,0.000000,0.000000
41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
562205,319.916626,0.000000,0.000000,0.000000,0.000000,0.000000,1648.989708
562312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
msumm = transactions.pivot_table(values="sm",
    index="user_id",
    columns="m",
    aggfunc='sum',
    fill_value=0,)
msumm.columns = list( map(lambda x: "msumm"+str(x), msumm.columns ) )
msumm

,msumm3,msumm4,msumm5,msumm6,msumm7,msumm8,msumm9
user_id,,,,,,,
3,0.000000,0.000000,-15144.601562,-153866.890625,-3390.320969,0.000000,0.000000
9,-13289.485764,-104223.187939,-109800.056656,-54146.384140,-18500.321304,-12387.577271,-11087.653740
13,-16394.193359,-27095.248047,-27650.769531,-76186.732422,-33908.388672,-53024.051758,-19224.679688
37,-32719.820328,-41364.700729,-67081.604048,-53322.076138,-69769.034760,-50056.823341,-28284.328693
41,-8045.445801,-17708.008820,-30354.633301,-17462.199585,-18175.117676,-16841.208984,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,-142.056610,0.000000,-2984.231926,-11080.026783,-15374.940796,0.000000
562205,-10452.779190,-7546.495855,-6601.707232,-4821.073084,-6082.155629,-5607.360139,-1349.243835
562312,0.000000,-2644.634823,-1362.274536,-3165.544842,-3938.301605,-4495.630554,-2931.434910


In [14]:
msum = transactions.pivot_table(values="transaction_amt",
    index="user_id",
    columns=["m","currency_rk"],
    #aggfunc='sum',
    aggfunc='count',
    fill_value=0,)
msum.columns = list( map(lambda x: "msum"+str(x), msum.columns ) )
msum

,"msum(3, 0)","msum(3, 1)","msum(3, 2)","msum(3, 3)","msum(4, 0)","msum(4, 1)","msum(4, 2)","msum(4, 3)","msum(5, 0)","msum(5, 1)",...,"msum(7, 2)","msum(7, 3)","msum(8, 0)","msum(8, 1)","msum(8, 2)","msum(8, 3)","msum(9, 0)","msum(9, 1)","msum(9, 2)","msum(9, 3)"
user_id,,,,,,,,,,,,,,,,,,,,,
3,0,3,0,0,0,3,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,0,8,0,0,0,15,0,0,0,9,...,0,0,0,9,0,0,0,14,0,0
13,0,2,0,0,0,3,0,0,0,2,...,0,0,1,3,0,0,0,1,0,0
37,0,15,0,0,0,77,0,0,0,67,...,0,0,0,37,0,0,0,10,0,0
41,0,2,0,0,0,4,0,0,0,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0,0,0,0,0,1,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
562205,0,12,0,0,0,29,0,0,0,33,...,0,0,0,23,0,0,0,8,0,0
562312,0,0,0,0,0,10,0,0,0,5,...,0,0,0,14,0,0,0,7,0,0


In [15]:
smc = transactions.pivot_table(values="sm",
    index="user_id",
    columns=["m","currency_rk"],
    aggfunc='count',
    fill_value=0,)
smc.columns = list( map(lambda x: "smc"+str(x), smc.columns ) )
smc

,"smc(3, 0)","smc(3, 1)","smc(3, 2)","smc(3, 3)","smc(4, 0)","smc(4, 1)","smc(4, 2)","smc(4, 3)","smc(5, 0)","smc(5, 1)",...,"smc(7, 2)","smc(7, 3)","smc(8, 0)","smc(8, 1)","smc(8, 2)","smc(8, 3)","smc(9, 0)","smc(9, 1)","smc(9, 2)","smc(9, 3)"
user_id,,,,,,,,,,,,,,,,,,,,,
3,0,3,0,0,0,3,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,0,8,0,0,0,15,0,0,0,9,...,0,0,0,9,0,0,0,14,0,0
13,0,2,0,0,0,3,0,0,0,2,...,0,0,1,3,0,0,0,1,0,0
37,0,15,0,0,0,77,0,0,0,67,...,0,0,0,37,0,0,0,10,0,0
41,0,2,0,0,0,4,0,0,0,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0,0,0,0,0,1,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
562205,0,12,0,0,0,29,0,0,0,33,...,0,0,0,23,0,0,0,8,0,0
562312,0,0,0,0,0,10,0,0,0,5,...,0,0,0,14,0,0,0,7,0,0


In [16]:
spc = transactions.pivot_table(values="sp",
    index="user_id",
    columns="m",
    aggfunc='count',
    fill_value=0,)
spc.columns = list( map(lambda x: "spc"+str(x), spc.columns ) )
spc

,spc3,spc4,spc5,spc6,spc7,spc8,spc9
user_id,,,,,,,
3,3,3,2,1,2,0,0
9,8,15,9,23,12,9,14
13,2,3,2,6,4,4,1
37,15,77,67,54,55,37,10
41,2,4,3,4,2,1,0
...,...,...,...,...,...,...,...
562043,0,1,0,5,27,4,0
562205,12,29,33,20,26,23,8
562312,0,10,5,6,14,14,7


In [17]:
spdc = transactions.pivot_table(values="sp",
    index="user_id",
    columns="dweek",
    aggfunc='count',
    fill_value=0,)
spdc.columns = list( map(lambda x: "spdc"+str(x), spdc.columns ) )
spdc

spds = transactions.pivot_table(values="sp",
    index="user_id",
    columns="dweek",
    aggfunc='sum',
    fill_value=0,)
spds.columns = list( map(lambda x: "spds"+str(x), spds.columns ) )
spds

,spds0,spds1,spds2,spds3,spds4,spds5,spds6
user_id,,,,,,,
3,5083.818176,0.000000,0.000000,28335.613281,37991.929688,114696.868652,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13,20843.610840,10952.112305,0.000000,0.000000,96970.961182,0.000000,0.000000
37,0.000000,0.000000,0.000000,5487.140625,0.000000,5251.647949,0.000000
41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
562043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
562205,0.000000,0.000000,0.000000,53.528648,1595.461060,319.916626,0.000000
562312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
smdc = transactions.pivot_table(values="sm",
    index="user_id",
    columns="dweek",
    aggfunc='count',
    fill_value=0,)
smdc.columns = list( map(lambda x: "smdc"+str(x), smdc.columns ) )
smdc

smds = transactions.pivot_table(values="sm",
    index="user_id",
    columns="dweek",
    aggfunc='sum',
    fill_value=0,)
smds.columns = list( map(lambda x: "smds"+str(x), smds.columns ) )
smds

,smds0,smds1,smds2,smds3,smds4,smds5,smds6
user_id,,,,,,,
3,0.000000,0.000000,-169011.492188,0.000000,-3390.320969,0.000000,0.000000
9,-73575.006897,-22025.790817,-12068.200882,-5015.756996,-24209.228561,-170599.176193,-15941.506466
13,-22091.132812,-24650.388672,-52853.843750,-5997.509766,-96384.850586,-17878.054688,-33628.283203
37,-42904.638344,-79845.543495,-47002.623840,-56788.511864,-28587.469574,-43567.516178,-43902.084742
41,-18852.887085,-23463.530762,-28168.941437,-11565.626953,-24716.487305,-1819.140625,0.000000
...,...,...,...,...,...,...,...
562043,-2115.068794,-1778.833473,-1404.176968,-4581.644516,-15148.450806,-769.264641,-3783.816917
562205,-1928.830556,-3430.759820,-3260.953653,-5024.823685,-6419.355495,-16460.974468,-5935.117287
562312,-2670.059166,-1222.038811,-2914.708450,-4018.634880,-2457.707897,-3825.585869,-1429.086197


In [19]:
max_day_trans = transactions.groupby(['user_id'])[['diff_days']].max().rename(columns={"diff_days":"max_day_trans"})
max_day_trans

,max_day_trans
user_id,
3,214
9,283
13,282
37,283
41,256
...,...
562043,266
562205,280
562312,280


In [20]:
min_day_trans = transactions.groupby(['user_id']
            )[['diff_days']].min().rename(columns={"diff_days":"min_day_trans"})
min_day_trans

,min_day_trans
user_id,
3,108
9,102
13,114
37,104
41,103
...,...
562043,142
562205,102
562312,122


In [21]:
mcc_trans = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns="mcc_code",
    aggfunc="sum",
    fill_value=0,)
mcc_trans.columns = list( map(lambda x: "mcc"+str(x), mcc_trans.columns ) )
mcc_trans

,mcc0,mcc1,mcc2,mcc3,mcc4,mcc6,mcc7,mcc8,mcc9,mcc10,...,mcc372,mcc382,mcc389,mcc392,mcc407,mcc412,mcc413,mcc424,mcc434,mcc449
user_id,,,,,,,,,,,,,,,,,,,,,
3,186108.229797,0.000000,0.000000,-3390.320969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,-20556.742374,-8324.033737,-192534.730225,-1664.872650,-2037.417374,0.000000,-5204.533508,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.000000,0.000000,0.000000,-160818.399902,0.000000,0.000000,-5997.509766,0.000000,10805.421875,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,10738.788574,-35943.518551,-26588.514793,-214788.286194,-126.869576,-653.359955,0.000000,-1031.058945,-26518.769531,-924.123138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.000000,-740.059052,0.000000,-70219.408691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0.000000,-7254.742481,0.000000,-13020.519531,0.000000,-1303.106201,-786.304443,-870.556900,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562205,-10868.047920,-12028.663424,-1157.922058,0.000000,-129.403091,-6800.897217,0.000000,-2645.133282,0.000000,-770.019485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562312,0.000000,-6974.722969,-352.799313,0.000000,-1372.377075,-2114.061485,0.000000,-988.508408,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
mcc_transc = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns="mcc_code",
    aggfunc='count',
    fill_value=0,)
mcc_transc.columns = list( map(lambda x: "mccc"+str(x), mcc_transc.columns ) )
mcc_transc

,mccc0,mccc1,mccc2,mccc3,mccc4,mccc6,mccc7,mccc8,mccc9,mccc10,...,mccc372,mccc382,mccc389,mccc392,mccc407,mccc412,mccc413,mccc424,mccc434,mccc449
user_id,,,,,,,,,,,,,,,,,,,,,
3,7,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,27,11,5,3,2,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,14,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
37,2,97,129,28,1,2,0,7,1,4,...,0,0,0,0,0,0,0,0,0,0
41,0,2,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0,19,0,1,0,1,3,4,0,0,...,0,0,0,0,0,0,0,0,0,0
562205,30,33,2,0,1,10,0,6,0,3,...,0,0,0,0,0,0,0,0,0,0
562312,0,24,3,0,1,6,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
mcc_transmc = transactions.pivot_table(    
    values="transaction_amt",
    index="user_id",
    columns=["mcc_code","m"],
    aggfunc=['sum','median'],
    fill_value=0,
)
mcc_transmc.columns = list( map(lambda x: "mccmc"+str(x), mcc_transmc.columns ) )
mcc_transmc

,"mccmc('sum', 0, 3)","mccmc('sum', 0, 4)","mccmc('sum', 0, 5)","mccmc('sum', 0, 6)","mccmc('sum', 0, 7)","mccmc('sum', 0, 8)","mccmc('sum', 0, 9)","mccmc('sum', 1, 3)","mccmc('sum', 1, 4)","mccmc('sum', 1, 5)",...,"mccmc('median', 412, 8)","mccmc('median', 413, 3)","mccmc('median', 413, 4)","mccmc('median', 413, 5)","mccmc('median', 413, 6)","mccmc('median', 413, 7)","mccmc('median', 413, 9)","mccmc('median', 424, 6)","mccmc('median', 434, 4)","mccmc('median', 449, 7)"
user_id,,,,,,,,,,,,,,,,,,,,,
3,137734.573242,43075.747864,5297.908691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5323.413376,-2728.293545,-172.254761,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0.000000,0.000000,5487.140625,5251.647949,0.000000,0.000000,0.000000,-2098.237946,-7730.614010,-4955.007622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-290.766998,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562205,-7609.368988,-1207.410591,-1161.791405,-568.072906,-1511.607269,-458.786469,1648.989708,0.000000,-108.493008,-2985.051830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1993.233418,-353.968170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
%%time
date_trans = transactions.groupby("user_id").agg({"date":"unique"}).explode("date").reset_index()#.sort(["user_id","date"])
date_trans["interval"]=date_trans.groupby("user_id")["date"].diff()
date_trans = date_trans.dropna() 
date_trans["interval"]=date_trans["interval"].dt.days
date_trans = date_trans.groupby("user_id").agg({"interval":["last","max"]})
date_trans.columns = list( map(lambda x: "date_"+str(x), date_trans.columns ) )
date_trans

CPU times: user 34.6 s, sys: 1.12 s, total: 35.7 s
Wall time: 35.7 s


,"date_('interval', 'last')","date_('interval', 'max')"
user_id,,
3,9,30
9,17,22
13,2,21
37,1,7
41,2,31
...,...,...
562043,54,54
562205,1,8
562312,2,37


# Итоговая сборка

In [25]:
df = clients.join(
                count_trans, on="user_id", how="left"
                ).join(temp_tran, on="user_id", how="left"
                ).join(msump, on="user_id", how="left"
                ).join(msumm, on="user_id", how="left"
                ).join(msum, on="user_id", how="left"
                ).join(spc, on="user_id", how="left"
                ).join(smc, on="user_id", how="left"
                ).join(mcc_trans, on="user_id", how="left"
                ).join(mcc_transc, on="user_id", how="left"
                ).join(date_trans, on="user_id", how="left"
                ).join(train, on="user_id", how="left"
                )

df['time'] = df['time'].fillna(-1)
df['time'] = df['time'].astype(np.int32)
df['target'] = df['target'].fillna(-1)
df['target'] = df['target'].astype(np.int8)

df

,report,employee_count_nm,bankemplstatus,customer_age,report_dt,count_trans_transaction_amt_sum,count_trans_transaction_amt_max,count_trans_transaction_amt_min,count_trans_transaction_amt_count,count_trans_mcc_code_nunique,...,mccc407,mccc412,mccc413,mccc424,mccc434,mccc449,"date_('interval', 'last')","date_('interval', 'max')",target,time
user_id,,,,,,,,,,,,,,,,,,,,,
3,2,4.0,0,3,2022-08-31 03:00:00,13706.416641,104011.960938,-153866.890625,11,4,...,0,0,0,0,0,0,9.0,30.0,0,77
9,1,6.0,0,3,2022-07-31 03:00:00,-323434.666813,-45.579891,-90147.617188,90,22,...,0,0,0,0,0,0,17.0,22.0,-1,-1
13,6,5.0,0,2,2022-12-31 03:00:00,-124717.379150,70322.828125,-58740.300781,22,4,...,0,0,0,0,0,0,2.0,21.0,0,86
37,5,6.0,0,2,2022-11-30 03:00:00,-331859.599463,5487.140625,-35782.984375,315,28,...,0,0,0,0,0,0,1.0,7.0,0,89
41,1,4.0,0,2,2022-07-31 03:00:00,-108586.614166,-290.766998,-16841.208984,16,5,...,0,0,0,0,0,0,2.0,31.0,0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562043,12,NaN,0,2,2023-06-30 03:00:00,-29581.256115,-40.769005,-13020.519531,37,12,...,0,0,0,0,0,0,54.0,54.0,0,75
562205,12,NaN,0,1,2023-06-30 03:00:00,-40491.908630,1595.461060,-6220.171387,151,15,...,0,0,0,0,0,0,1.0,8.0,-1,-1
562312,12,NaN,0,0,2023-06-30 03:00:00,-18537.821270,-28.292030,-1372.377075,56,7,...,0,0,0,0,0,0,2.0,37.0,0,91


# train_test_split

In [26]:
X = df[df['time']!=-1].copy()
y = df[df['time']!=-1][['target']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,report,employee_count_nm,bankemplstatus,customer_age,report_dt,count_trans_transaction_amt_sum,count_trans_transaction_amt_max,count_trans_transaction_amt_min,count_trans_transaction_amt_count,count_trans_mcc_code_nunique,...,mccc407,mccc412,mccc413,mccc424,mccc434,mccc449,"date_('interval', 'last')","date_('interval', 'max')",target,time
user_id,,,,,,,,,,,,,,,,,,,,,
379659,12,NaN,0,3,2023-06-30 03:00:00,-4.912574e+05,173.600006,-37796.332031,261,28,...,0,0,0,0,0,0,1.0,8.0,0,91
317004,5,NaN,0,1,2022-11-30 03:00:00,-1.600682e+06,94616.492188,-161607.062500,259,31,...,0,0,0,0,0,0,2.0,5.0,0,90
329546,3,NaN,0,3,2022-09-30 03:00:00,-3.966895e+05,-3778.484375,-35181.617188,29,1,...,0,0,0,0,0,0,4.0,42.0,0,67
16918,2,2.0,0,3,2022-08-31 03:00:00,-4.363860e+05,12789.578125,-85379.859375,25,5,...,0,0,0,0,0,0,32.0,32.0,0,89
246698,1,NaN,0,1,2022-07-31 03:00:00,9.613582e+03,16012.974609,-8326.995117,143,20,...,0,0,0,0,0,0,2.0,22.0,0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538772,12,NaN,0,0,2023-06-30 03:00:00,2.135669e+03,4878.644043,-613.594727,28,3,...,0,0,0,0,0,0,2.0,24.0,0,75
330596,10,4.0,0,2,2023-04-30 03:00:00,7.873303e+04,37352.253906,-18147.533203,64,11,...,0,0,0,0,0,0,1.0,21.0,0,91
7334,3,4.0,0,3,2022-09-30 03:00:00,-8.045764e+05,200906.968750,-109847.617188,153,31,...,0,0,0,0,0,0,1.0,18.0,0,91


In [27]:
from sklearn.model_selection import StratifiedKFold

In [32]:
X_train = X_train.drop(['target', 'time'], axis=1)

In [37]:
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
frames_for_metamodel = []
metadata = None
cats = []
for i in range(5):
    frames_for_metamodel = []
    for train_index, valid_index in strat_kfold.split(X_train, y_train):
        X_train_, X_val = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_, y_val = y_train.iloc[train_index], y_train.iloc[valid_index]
        cat = CatBoostClassifier(
            iterations=2500,    
            learning_rate= 0.03, 
            depth=5,
            eval_metric="AUC", 
            random_seed=63+9+i, 
            task_type="GPU",
        )
        cat_features = ["bankemplstatus","customer_age"]
        cat.fit(X_train_,y_train_, eval_set=(X_val, y_val), cat_features=cat_features, verbose=500, plot=True)
        print(i, cat.best_score_)
        cats.append(cat)
        predict = cat.predict_proba(X_val)[:,1]
        frames_for_metamodel.append(pd.DataFrame({'user_id': X_val.index.values, 'pred_agg_0010': predict}))
    if metadata is not None:
        metadata['pred_agg_new'] += pd.concat(frames_for_metamodel, axis=0).reset_index(drop=True)['pred_agg_0010']
    else:
        metadata = pd.concat(frames_for_metamodel, axis=0).reset_index()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6428828	best: 0.6428828 (0)	total: 37.1ms	remaining: 1m 32s
500:	test: 0.7629238	best: 0.7629238 (500)	total: 16.9s	remaining: 1m 7s
1000:	test: 0.7656560	best: 0.7658063 (930)	total: 33.8s	remaining: 50.6s
1500:	test: 0.7658731	best: 0.7661613 (1385)	total: 50.5s	remaining: 33.6s
2000:	test: 0.7649183	best: 0.7661613 (1385)	total: 1m 7s	remaining: 16.8s
2499:	test: 0.7644121	best: 0.7661613 (1385)	total: 1m 24s	remaining: 0us
bestTest = 0.766161263
bestIteration = 1385
Shrink model to first 1386 iterations.
0 {'learn': {'Logloss': 0.21634383201599122}, 'validation': {'Logloss': 0.24766056537628173, 'AUC': 0.7661612629890442}}


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


...

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5591322	best: 0.5591322 (0)	total: 33.5ms	remaining: 1m 23s
500:	test: 0.7583126	best: 0.7583126 (500)	total: 16.7s	remaining: 1m 6s
1000:	test: 0.7629868	best: 0.7629972 (990)	total: 33.5s	remaining: 50.2s
1500:	test: 0.7646019	best: 0.7646640 (1495)	total: 50.2s	remaining: 33.4s
2000:	test: 0.7653214	best: 0.7653447 (1985)	total: 1m 6s	remaining: 16.7s
2499:	test: 0.7663801	best: 0.7665030 (2480)	total: 1m 23s	remaining: 0us
bestTest = 0.766503036
bestIteration = 2480
Shrink model to first 2481 iterations.
4 {'learn': {'Logloss': 0.21477649211883545}, 'validation': {'Logloss': 0.24644057750701903, 'AUC': 0.7665030360221863}}


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6844294	best: 0.6844294 (0)	total: 30.6ms	remaining: 1m 16s
500:	test: 0.7577600	best: 0.7577807 (495)	total: 16.7s	remaining: 1m 6s
1000:	test: 0.7616196	best: 0.7616196 (1000)	total: 33.7s	remaining: 50.4s
1500:	test: 0.7614548	best: 0.7616622 (1400)	total: 50.2s	remaining: 33.4s
2000:	test: 0.7615258	best: 0.7617217 (1555)	total: 1m 6s	remaining: 16.7s
2499:	test: 0.7611256	best: 0.7617217 (1555)	total: 1m 23s	remaining: 0us
bestTest = 0.7617217302
bestIteration = 1555
Shrink model to first 1556 iterations.
4 {'learn': {'Logloss': 0.21680693626403807}, 'validation': {'Logloss': 0.24884397983551027, 'AUC': 0.7617217302322388}}


In [49]:
predict_test = np.zeros(X_test.shape[0])
predict_real = np.zeros(df[df['time']==-1][df.columns[:-2]].shape[0])

for cat in cats:
    predict_test += cat.predict_proba(X_test.drop(['target', 'time'], axis=1))[:, 1]
    predict_real += cat.predict_proba(df[df['time']==-1][df.columns[:-2]])[:, 1]

In [51]:
pred_test = {'user_id': X_test.index.values, 'pred_agg_0010': predict_test}
pred_real = {'user_id': df[df['time']==-1][df.columns[:-2]].index.values, 'pred_agg_0010': predict_real}

metadata = pd.concat([metadata, test_pred, real_pred])
metadata['pred_agg_0010'] = metadata['pred_agg_0010']/25
metadata.to_csv('agg_0010_meta.csv', index=False)

In [ ]:
embeddings_path = f'../output_embs/coles_emb.csv'
main_embs = pd.read_csv(embeddings_path)

wtte_embeddings_path = f'../output_embs/wtte_embs.csv'
wtte_embs = pd.read_csv(wtte_embeddings_path)
main_embs = main_embs.merge(wtte_embs, on='user_id')

In [31]:
X_train['user_id'] = X_train.index
X_train.reset_index(drop=True, inplace=True)
X_train_emb = X_train.merge(main_embs, on='user_id')

In [38]:
cat_features = ["bankemplstatus","customer_age"]

In [39]:
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X_, y_ = X_train_emb.drop(['time', 'target','report_dt'], axis=1), X_train_emb['time']
time_models = []
for train_index, valid_index in strat_kfold.split(X_train_emb, X_train_emb['target']):
    
    X_train_, X_val = X_.iloc[train_index], X_.iloc[valid_index]
    y_train_, y_val = y_.iloc[train_index], y_.iloc[valid_index]

    model = CatBoostRegressor(
        iterations = 2500,
        depth=5,
        learning_rate=0.02,
        cat_features = cat_features,
        early_stopping_rounds=400,
        random_seed = 42,
        eval_metric='MSLE',
        task_type="GPU",
    )

    model.fit(Pool(X_train_, y_train_, cat_features=cat_features),
              eval_set=Pool(X_val, y_val, cat_features=cat_features),
              verbose=100)
    time_models.append(model)

Default metric period is 5 because MSLE is/are not implemented for GPU
Metric MSLE is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.1766289	test: 0.1733291	best: 0.1733291 (0)	total: 39.1ms	remaining: 1m 37s
100:	learn: 0.1626520	test: 0.1602740	best: 0.1602740 (100)	total: 3.47s	remaining: 1m 22s
200:	learn: 0.1598668	test: 0.1581236	best: 0.1581236 (200)	total: 6.76s	remaining: 1m 17s
300:	learn: 0.1586156	test: 0.1572689	best: 0.1572689 (300)	total: 10s	remaining: 1m 13s
400:	learn: 0.1578604	test: 0.1568862	best: 0.1568862 (399)	total: 13.2s	remaining: 1m 8s
500:	learn: 0.1572679	test: 0.1566747	best: 0.1566747 (500)	total: 16.4s	remaining: 1m 5s
600:	learn: 0.1565247	test: 0.1563819	best: 0.1563805 (598)	total: 19.6s	remaining: 1m 1s
700:	learn: 0.1558643	test: 0.1562024	best: 0.1562011 (699)	total: 22.9s	remaining: 58.7s
800:	learn: 0.1552368	test: 0.1560156	best: 0.1560155 (799)	total: 26s	remaining: 55.2s
900:	learn: 0.1546808	test: 0.1559148	best: 0.1559148 (900)	total: 29.2s	remaining: 51.8s
1000:	learn: 0.1540496	test: 0.1558023	best: 0.1558023 (1000)	total: 32.4s	remaining: 48.5s
1100:	learn

Default metric period is 5 because MSLE is/are not implemented for GPU
Metric MSLE is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.1772297	test: 0.1713912	best: 0.1713912 (0)	total: 49.3ms	remaining: 2m 3s
100:	learn: 0.1633580	test: 0.1575469	best: 0.1575469 (100)	total: 3.68s	remaining: 1m 27s
200:	learn: 0.1605921	test: 0.1553172	best: 0.1553172 (200)	total: 7.17s	remaining: 1m 22s
300:	learn: 0.1593373	test: 0.1545512	best: 0.1545512 (300)	total: 10.6s	remaining: 1m 17s
400:	learn: 0.1585257	test: 0.1541637	best: 0.1541637 (400)	total: 14s	remaining: 1m 13s
500:	learn: 0.1577791	test: 0.1538410	best: 0.1538410 (500)	total: 17.4s	remaining: 1m 9s
600:	learn: 0.1571303	test: 0.1536584	best: 0.1536567 (599)	total: 20.8s	remaining: 1m 5s
700:	learn: 0.1565246	test: 0.1535157	best: 0.1535128 (697)	total: 24.2s	remaining: 1m 2s
800:	learn: 0.1559046	test: 0.1533461	best: 0.1533431 (794)	total: 27.6s	remaining: 58.6s
900:	learn: 0.1554074	test: 0.1532519	best: 0.1532519 (900)	total: 31s	remaining: 55s
1000:	learn: 0.1548271	test: 0.1531904	best: 0.1531891 (999)	total: 34.3s	remaining: 51.4s
1100:	learn: 0

Default metric period is 5 because MSLE is/are not implemented for GPU
Metric MSLE is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

Default metric period is 5 because MSLE is/are not implemented for GPU
Metric MSLE is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.1781152	test: 0.1673469	best: 0.1673469 (0)	total: 40.9ms	remaining: 1m 42s
100:	learn: 0.1638327	test: 0.1551497	best: 0.1551497 (100)	total: 3.78s	remaining: 1m 29s
200:	learn: 0.1609137	test: 0.1530000	best: 0.1530000 (200)	total: 7.34s	remaining: 1m 23s
300:	learn: 0.1596464	test: 0.1522920	best: 0.1522920 (300)	total: 10.8s	remaining: 1m 18s
400:	learn: 0.1588173	test: 0.1519504	best: 0.1519504 (400)	total: 14.2s	remaining: 1m 14s
500:	learn: 0.1581065	test: 0.1517394	best: 0.1517394 (500)	total: 17.7s	remaining: 1m 10s
600:	learn: 0.1574327	test: 0.1515444	best: 0.1515444 (600)	total: 21.1s	remaining: 1m 6s
700:	learn: 0.1568177	test: 0.1514525	best: 0.1514522 (699)	total: 24.5s	remaining: 1m 2s
800:	learn: 0.1562274	test: 0.1513620	best: 0.1513620 (800)	total: 27.8s	remaining: 59s
900:	learn: 0.1556415	test: 0.1512516	best: 0.1512516 (900)	total: 31.4s	remaining: 55.7s
1000:	learn: 0.1551066	test: 0.1511730	best: 0.1511730 (1000)	total: 34.8s	remaining: 52s
1100:	lea

Default metric period is 5 because MSLE is/are not implemented for GPU
Metric MSLE is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.1766382	test: 0.1736901	best: 0.1736901 (0)	total: 40.1ms	remaining: 1m 40s
100:	learn: 0.1626996	test: 0.1605316	best: 0.1605316 (100)	total: 3.69s	remaining: 1m 27s
200:	learn: 0.1598277	test: 0.1582016	best: 0.1582016 (200)	total: 7.22s	remaining: 1m 22s
300:	learn: 0.1584717	test: 0.1573771	best: 0.1573771 (300)	total: 10.8s	remaining: 1m 18s
400:	learn: 0.1577064	test: 0.1570235	best: 0.1570234 (399)	total: 14.2s	remaining: 1m 14s
500:	learn: 0.1570439	test: 0.1567683	best: 0.1567683 (500)	total: 17.7s	remaining: 1m 10s
600:	learn: 0.1563796	test: 0.1566437	best: 0.1566427 (595)	total: 21.1s	remaining: 1m 6s
700:	learn: 0.1557156	test: 0.1564732	best: 0.1564718 (698)	total: 24.5s	remaining: 1m 2s
800:	learn: 0.1551220	test: 0.1564083	best: 0.1564083 (800)	total: 28s	remaining: 59.5s
900:	learn: 0.1545371	test: 0.1563118	best: 0.1563095 (891)	total: 31.5s	remaining: 56s
1000:	learn: 0.1540341	test: 0.1562594	best: 0.1562518 (981)	total: 35s	remaining: 52.4s
1100:	learn:

/home/jovyan/.local/share/virtualenvs/ptls-experiments-oubtHWvK/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jovyan/.local/share/virtualenvs/ptls-experiments-oubtHWvK/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [44]:
scores = []
frames_for_metamodel = []
models = []
for i, (train_index, valid_index) in enumerate(strat_kfold.split(X_train_emb, X_train_emb['target'])):
    train_with_embs_churn = X_train_emb.drop(['time', 'target'], axis=1).copy()
    train_with_embs_churn['new_time'] = time_models[i].predict(X_train_emb)
    X_, y_ = train_with_embs_churn, X_train_emb['target']
    X_train_, X_val = X_.iloc[train_index], X_.iloc[valid_index]
    y_train_, y_val = y_.iloc[train_index], y_.iloc[valid_index]

    model = CatBoostClassifier(
        iterations = 6500,
        depth=4,
        learning_rate=0.005,
        eval_metric='AUC',
        cat_features = cat_features,
        early_stopping_rounds=400,
        task_type="GPU",
    )

    model.fit(Pool(X_train_, y_train_, cat_features=cat_features),
              eval_set=Pool(X_val, y_val, cat_features=cat_features),
              verbose=100)
    models.append(model)
    
    pred = model.predict_proba(X_val)[:, 1]
    frames_for_metamodel.append(pd.DataFrame({'user_id': X_val.user_id.values, 'pred_time_0010': pred}))
    scores.append(metrics.roc_auc_score(y_val, pred))

print(np.mean(scores))
metadata = pd.concat(frames_for_metamodel, axis=0).reset_index(drop=True)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5845847	best: 0.5845847 (0)	total: 40.4ms	remaining: 4m 22s
100:	test: 0.7472633	best: 0.7473426 (99)	total: 4.01s	remaining: 4m 13s
200:	test: 0.7529712	best: 0.7529740 (198)	total: 7.98s	remaining: 4m 10s
300:	test: 0.7564597	best: 0.7564597 (300)	total: 12s	remaining: 4m 7s
400:	test: 0.7600823	best: 0.7601380 (398)	total: 16s	remaining: 4m 3s
500:	test: 0.7632804	best: 0.7632804 (500)	total: 20s	remaining: 3m 59s
600:	test: 0.7646973	best: 0.7647249 (598)	total: 24s	remaining: 3m 55s
700:	test: 0.7669637	best: 0.7669637 (700)	total: 28s	remaining: 3m 51s
800:	test: 0.7680403	best: 0.7680403 (800)	total: 31.9s	remaining: 3m 46s
900:	test: 0.7690418	best: 0.7690459 (899)	total: 36s	remaining: 3m 43s
1000:	test: 0.7698905	best: 0.7698958 (998)	total: 40s	remaining: 3m 39s
1100:	test: 0.7705095	best: 0.7705230 (1098)	total: 44.1s	remaining: 3m 36s
1200:	test: 0.7710757	best: 0.7710787 (1195)	total: 48s	remaining: 3m 31s
1300:	test: 0.7716255	best: 0.7716255 (1300)	total: 52.

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6127818	best: 0.6127818 (0)	total: 40.5ms	remaining: 4m 23s
100:	test: 0.7425406	best: 0.7425406 (100)	total: 3.93s	remaining: 4m 8s
200:	test: 0.7465638	best: 0.7465638 (200)	total: 7.86s	remaining: 4m 6s
300:	test: 0.7496333	best: 0.7496333 (300)	total: 11.8s	remaining: 4m 2s
400:	test: 0.7526944	best: 0.7526944 (400)	total: 15.8s	remaining: 3m 59s
500:	test: 0.7548104	best: 0.7548104 (500)	total: 19.7s	remaining: 3m 55s
600:	test: 0.7566648	best: 0.7566797 (598)	total: 23.7s	remaining: 3m 52s
700:	test: 0.7579426	best: 0.7579426 (700)	total: 27.8s	remaining: 3m 49s
800:	test: 0.7588357	best: 0.7588482 (799)	total: 31.6s	remaining: 3m 45s
900:	test: 0.7596089	best: 0.7596406 (898)	total: 35.7s	remaining: 3m 41s
1000:	test: 0.7601928	best: 0.7602021 (999)	total: 39.7s	remaining: 3m 38s
1100:	test: 0.7607327	best: 0.7607377 (1099)	total: 43.8s	remaining: 3m 34s
1200:	test: 0.7611802	best: 0.7611803 (1199)	total: 47.8s	remaining: 3m 30s
1300:	test: 0.7617353	best: 0.7617353 (

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5847802	best: 0.5847802 (0)	total: 41.6ms	remaining: 4m 30s
100:	test: 0.7179728	best: 0.7181994 (96)	total: 3.88s	remaining: 4m 5s
200:	test: 0.7232990	best: 0.7234561 (197)	total: 7.91s	remaining: 4m 7s
300:	test: 0.7269645	best: 0.7270143 (276)	total: 11.8s	remaining: 4m 3s
400:	test: 0.7313880	best: 0.7313880 (400)	total: 15.9s	remaining: 4m 1s
500:	test: 0.7336203	best: 0.7336526 (496)	total: 19.9s	remaining: 3m 58s
600:	test: 0.7364714	best: 0.7364714 (600)	total: 24.1s	remaining: 3m 56s
700:	test: 0.7388380	best: 0.7388380 (700)	total: 28.2s	remaining: 3m 53s
800:	test: 0.7406660	best: 0.7406660 (800)	total: 32.2s	remaining: 3m 49s
900:	test: 0.7422664	best: 0.7422664 (900)	total: 36.2s	remaining: 3m 45s
1000:	test: 0.7437404	best: 0.7437404 (1000)	total: 40.4s	remaining: 3m 41s
1100:	test: 0.7448189	best: 0.7448189 (1100)	total: 44.4s	remaining: 3m 37s
1200:	test: 0.7457451	best: 0.7457451 (1200)	total: 48.4s	remaining: 3m 33s
1300:	test: 0.7464111	best: 0.7464111 (1

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5864719	best: 0.5864719 (0)	total: 41.5ms	remaining: 4m 29s
100:	test: 0.7335256	best: 0.7335741 (99)	total: 4.03s	remaining: 4m 15s
200:	test: 0.7392248	best: 0.7392855 (199)	total: 8.07s	remaining: 4m 13s
300:	test: 0.7449483	best: 0.7449483 (300)	total: 12.1s	remaining: 4m 8s
400:	test: 0.7492989	best: 0.7492989 (400)	total: 16.1s	remaining: 4m 4s
500:	test: 0.7521583	best: 0.7522016 (499)	total: 20.2s	remaining: 4m 1s
600:	test: 0.7535276	best: 0.7535276 (600)	total: 24.3s	remaining: 3m 58s
700:	test: 0.7551016	best: 0.7551016 (700)	total: 28.3s	remaining: 3m 54s
800:	test: 0.7568285	best: 0.7568285 (800)	total: 32.3s	remaining: 3m 49s
900:	test: 0.7580963	best: 0.7580963 (900)	total: 36.3s	remaining: 3m 45s
1000:	test: 0.7589117	best: 0.7589117 (1000)	total: 40.3s	remaining: 3m 41s
1100:	test: 0.7597505	best: 0.7597505 (1100)	total: 44.2s	remaining: 3m 36s
1200:	test: 0.7603952	best: 0.7603952 (1200)	total: 48.2s	remaining: 3m 32s
1300:	test: 0.7609277	best: 0.7609277 (

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6022945	best: 0.6022945 (0)	total: 40.1ms	remaining: 4m 20s
100:	test: 0.7362552	best: 0.7364490 (99)	total: 4.02s	remaining: 4m 14s
200:	test: 0.7422773	best: 0.7423698 (197)	total: 8.09s	remaining: 4m 13s
300:	test: 0.7458244	best: 0.7458244 (300)	total: 12s	remaining: 4m 7s
400:	test: 0.7484292	best: 0.7487130 (378)	total: 16s	remaining: 4m 2s
500:	test: 0.7497440	best: 0.7498172 (496)	total: 19.9s	remaining: 3m 58s
600:	test: 0.7513004	best: 0.7513203 (599)	total: 23.9s	remaining: 3m 54s
700:	test: 0.7527240	best: 0.7527274 (695)	total: 27.9s	remaining: 3m 50s
800:	test: 0.7542723	best: 0.7542723 (800)	total: 31.9s	remaining: 3m 46s
900:	test: 0.7552892	best: 0.7552892 (900)	total: 35.9s	remaining: 3m 42s
1000:	test: 0.7561291	best: 0.7561291 (1000)	total: 39.9s	remaining: 3m 39s
1100:	test: 0.7566674	best: 0.7566708 (1098)	total: 43.8s	remaining: 3m 34s
1200:	test: 0.7572267	best: 0.7572364 (1198)	total: 47.8s	remaining: 3m 30s
1300:	test: 0.7577178	best: 0.7577178 (130

In [186]:
metadata = metadata.drop('index', axis=1)
metadata.to_csv('time_pred_newagg_800_alfabetanew3_meta.csv', index=False)

In [46]:
X_test['user_id'] = X_test.index
X_test.reset_index(drop=True, inplace=True)
X_test_emb = X_test.merge(main_embs, on='user_id')

In [47]:
from sklearn import metrics
X_test_ = X_test_emb.drop(['report_dt', 'target', 'time',], axis=1)
time =[time_models[i].predict(X_test_) for i in range(5)]
X_test_['new_time'] = np.mean(time, axis =0)
predict = np.zeros(X_test_.shape[0])
for i in range(len(models)):
    predict += models[i].predict_proba(X_test_)[:, 1]
print(metrics.roc_auc_score(X_test['target'], predict))

0.7795979423299644


In [ ]:
#0.781 

In [190]:
test_pred = {'user_id': X_test_emb.user_id.values, 'pred_time_0010': predict}

In [ ]:
X_ = X.copy()
X_['user_id'] = X_.index
X_.reset_index(drop=True, inplace=True)
X_train_emb = X_.merge(main_embs, on='user_id')

In [ ]:
scores = []
models = []
for i, (train_index, valid_index) in enumerate(strat_kfold.split(X_train_emb, X_train_emb['target'])):
    train_with_embs_churn = X_train_emb.drop(['time', 'target'], axis=1).copy()
    train_with_embs_churn['new_time'] = time_models[i].predict(X_train_emb)
    X_, y_ = train_with_embs_churn, X_train_emb['target']
    X_train_, X_val = X_.iloc[train_index], X_.iloc[valid_index]
    y_train_, y_val = y_.iloc[train_index], y_.iloc[valid_index]

    model = CatBoostClassifier(
        iterations = 6500,
        depth=4,
        learning_rate=0.005,
        eval_metric='AUC',
        cat_features = cat_features,
        early_stopping_rounds=400,
        task_type="GPU",
    )

    model.fit(Pool(X_train_, y_train_, cat_features=cat_features),
              eval_set=Pool(X_val, y_val, cat_features=cat_features),
              verbose=100)
    models.append(model)
    pred = model.predict_proba(X_val)[:, 1]
    scores.append(metrics.roc_auc_score(y_val, pred))

print(np.mean(scores))
metadata = pd.concat(frames_for_metamodel, axis=0).reset_index(drop=True)

In [ ]:
X_real = df[df['time']==-1][df.columns[:-2]].copy()
X_real['user_id'] = X_real.index
X_real.reset_index(drop=True, inplace=True)
X_real_emb = X_real.merge(main_embs, on='user_id')
X_real_ = X_real_emb.drop(['report_dt', 'target', 'time',], axis=1)

time =[time_models[i].predict(X_real_) for i in range(5)]
X_real_['new_time'] = np.mean(time, axis =0)
predict = np.zeros(X_real_.shape[0])
for i in range(len(models)):
    predict += models[i].predict_proba(X_real_)[:, 1]

In [ ]:
predict = np.zeros(df[df['time']==-1][df.columns[:-2]].shape[0])
for i in range(len(models)):
    predict += models[i].predict_proba(df[df['time']==-1][df.columns[:-2]])[:, 1]

In [ ]:
real_pred = {'user_id': X_real_.user_id.values, 'pred_time_0010': predict}

metadata = pd.concat([metadata, test_pred, real_pred])
metadata['pred_time'] = metadata['pred_time']/5
metadata.to_csv('time_0010_pred_meta.csv', index=False)